# Herdnet Image Detection Demo with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife for image detection. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [1]:
import numpy as np
from PIL import Image
import torch
import torchvision

from torch.utils.data import DataLoader

from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife.data import datasets as pw_data 
from PytorchWildlife import utils as pw_utils

/home/lfvargas/anaconda3/envs/ptw/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model Initialization
We will initialize the MegaDetectorV5 model for image detection. This model is designed for detecting animals in images.

In [2]:
# Setting the device to use for computations ('cuda' indicates GPU)
CUDA_VISIBLE_DEVICES=1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
detection_model = pw_detection.HerdNetV1(device=DEVICE, pretrained=True)

## Single Image Detection
We will first perform detection on a single image. Make sure to verify that you have the image in the specified path.

In [3]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
detection_model = pw_detection.HerdNetV1(device=DEVICE, pretrained=True)

tgt_img_path = "./demo_data/herdimgs/A1_0718172_crop.JPG" 
img = np.array(Image.open(tgt_img_path).convert("RGB"))

# Performing the detection on the single image
results = detection_model.single_image_detection(img, tgt_img_path) 
pw_utils.save_detection_images(results, "./demo_output",annotation_type='point')

## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [4]:
tgt_folder_path = "/home/lfvargas/macaw/CameraTraps/demo/demo_data/herdimgs"
dataset = pw_data.DetectionImageFolder(
    tgt_folder_path,
    transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
    )

loader = DataLoader(dataset, batch_size=32, shuffle=False, 
                    pin_memory=True, num_workers=8, drop_last=False)
results = detection_model.batch_image_detection(loader)

2it [00:02,  1.05s/it]                       


## Output Results
PytorchWildlife allows to output detection results in multiple formats. Here are the examples:

### 1. Annotated Images:
This will output the images with bounding boxes drawn around the detected animals. The images will be saved in the specified output directory.

In [17]:
pw_utils.save_detection_images(results, "batch_output", annotation_type="point")

### 3. JSON Format:
This will output the detection results in JSON format. The JSON file will be saved in the specified output directory.

In [21]:
pw_utils.save_detection_json(results, "./batch_output.json",
                             categories=detection_model.CLASS_NAMES)

### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.